### one time installations

In [1]:
# !gdown --id 1pb7gEkctrVrJA79EAIo7H7nuzD6uV1fW
# !gdown --id 1oIeAE9HXXKWPcYa-AZ0ht5ef6sKe_Vh_
# !gdown --id 10rAuIDvsYR2yDiCqP7GmYGPc-UmtLbJb

In [2]:
# !pip install --quiet transformers
# !pip install --quiet datasets 
# !pip install --quiet SentencePiece
# !pip install --quiet pytorch-lightning

### libraries

In [3]:
import numpy as np
import pandas as pd
import sklearn
import random
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import Trainer, TrainingArguments

### hyperparameters 

In [14]:
class hyperparameters:
    # seed
    seed = 4
    
    # tokenizer
    tokenizer_name = "deepset/xlm-roberta-large-squad2" # model_name # CHANGE THIS; TRY XLM-ROBERTA
    max_len = 384 # maximum length of context and question in a datapoint
    overlap_len = 128 # overlap between two parts of the context when it is split
    
    # model
    model_name = "deepset/xlm-roberta-large-squad2"
    
    batch_size = 16

In [15]:
hyperparams = hyperparameters()

In [16]:
print("available gpu count:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(torch.cuda.device(i))

available gpu count: 1


### tokenizer

In [17]:
tokenizer = AutoTokenizer.from_pretrained(hyperparams.tokenizer_name)

### Raw (data) materials for Chaii

In [18]:
chaii_df = pd.read_csv('train.csv', encoding='utf-8')
# test_df = pd.read_csv('test.csv')
# sample_df = pd.read_csv('sample_submission.csv')
# chaii_df = sklearn.utils.shuffle(chaii_df, random_state=4).reset_index(drop=True)

In [19]:
train_df, val_df = model_selection.train_test_split(chaii_df, test_size=0.2, random_state=hyperparams.seed)

In [20]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [21]:
val_df

,id,context,question,answer_text,answer_start,language
0,5179ed725,ரேடியம் (Radium) என்பது Ra என்ற மூலக்கூற்று வா...,ரேடியம் எப்போது கண்டுபிடிக்கப்பட்டது?,1898,975,tamil
1,b028b54cf,हेलेन एडम्स केलर (27 जून 1880 - 1 जून 1968) एक...,हेलेन केलर की मृत्यु किस वर्ष में हुई थी?,1968,38,hindi
2,86eff66f2,फ़ूड एण्ड ड्रग एडमिनिस्ट्रेशन (FDA या USFDA) स...,खाद्य एवं औषधि प्रशासन का मुख्यालय कहाँ पर है?,"सिल्वर स्प्रिंग, मैरीलैंड",1092,hindi
3,6e76514a5,क़ुस्तुंतुनिया या कांस्टैंटिनोपुल (यूनानी: Κων...,क़ुस्तुंतुनिया शहर की स्थापना किस सम्राट ने की...,कोन्स्टान्टिन महान,573,hindi
4,9658a986b,दिल्ली समुद्रतल से ७०० से १००० फीट की ऊँचाई पर...,दिल्ली का क्षेत्रफल कितना है?,"१,४८४ वर्ग किलोमीटर",326,hindi
...,...,...,...,...,...,...
218,481ebd62d,Rzeczpospolita Polska Republic of Poland पोलैं...,पोलैंड का सबसे बड़ा शहर कौनसा है?,वारसा,192,hindi
219,934090840,தமிழ்த்திரையுலகின் முதல் பேசும்படம் \n1931 ஆம்...,ஹரிதாஸ் திரைப்படம் எந்த ஆண்டு வெளியானது?,1944,181,tamil
220,80a6914c2,மும்பை பங்குச் சந்தை (Bombay Stock Exchange) ஆ...,பாம்பே பங்குச் சந்தை எப்போது நிறுவப்பட்டது?,1875,147,tamil
221,88c1aa393,கலைக்களஞ்சியம் (ஒலிப்பு) (Encyclopedia)[1] என்...,டெனிசு டிடேரோ எந்த மொழியில் கலைக்களஞ்சியம் வெள...,பிரெஞ்சு,480,tamil


### Data pipeline

In [22]:
def prepare_chaii(data_df, tokenizer):
    # prepare_chaii takes in raw data and returns tokenized data 
    # along with position of first token and last token in the answer_text
    
    # strip trailing and leading whitespaces in context, question, and (answer_text)?
    data_df.loc[:, 'context'] = data_df.loc[:, 'context'].apply(lambda sen : str(sen).strip())
    data_df.loc[:, 'question'] = data_df.loc[:, 'question'].apply(lambda sen : str(sen).strip())
    data_df.loc[:, 'answer_text'] = data_df.loc[:, 'answer_text'].apply(lambda sen : str(sen).strip())
    
    data_tok = tokenizer(
        list(data_df['question']), list(data_df['context']),
        max_length=hyperparams.max_len, 
        truncation='only_second',
        stride=hyperparams.overlap_len,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=True,
    )
    
    # data_df contains original raw data having question, context
    # data_tok contains tokenized data, where context might have split into multiple sentences 
    # data_tok is a dict, containing keys : dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])
    # every value is a list, and no tensors here
    
    # adding two more keys that will contain the position of first token and last token in the answer_text
    data_tok['start_positions'], data_tok['end_positions'] = [], []
    
    n_sents = len(data_tok['input_ids'])
    map_id_sent2context = data_tok['overflow_to_sample_mapping'] # id means index! since input_ids means various inputs to the model
    map_offsets = data_tok['offset_mapping']
    assert len(map_offsets) == len(map_id_sent2context) == n_sents
    
    for input_id in range(n_sents):
        sent = data_tok['input_ids'][input_id]
        
        # get the answer_start and answer_text for this input_id using the id in data_df
        context_id = map_id_sent2context[input_id]
        answer_text = data_df.loc[context_id, 'answer_text']
        answer_start = data_df.loc[context_id, 'answer_start']
        answer_end = answer_start + len(answer_text) # will use this in next code block
        
        # check whether the answer is present in the current input_id or not using offsets
        qn_context_id = data_tok.sequence_ids(input_id)
        
            # first: get the start_idx_token and end_idx_token of context
        start_idx_token = qn_context_id.index(1)
        end_idx_token = len(qn_context_id) - qn_context_id[::-1].index(1) - 1
        
            # second: use the offsets for input_id to find if answer_start and answer_end are inside this chunk of context or not
        offset_map = map_offsets[input_id]

        if answer_start >= offset_map[start_idx_token][0] and answer_end <= offset_map[end_idx_token][1]:
            # now finally get the idx_token for the first and last token in the answer_text
            while answer_start >= offset_map[start_idx_token][0] and start_idx_token < len(sent):
                start_idx_token += 1
            while answer_end <= offset_map[end_idx_token][1]:
                end_idx_token -= 1
            
            data_tok['start_positions'].append(start_idx_token - 1)
            data_tok['end_positions'].append(end_idx_token + 1)
        
        else:
            cls_token_idx = sent.index(tokenizer.cls_token_id)
            assert cls_token_idx == 0
            data_tok['start_positions'].append(0) # cls token index
            data_tok['end_positions'].append(0) # cls token index

    return data_tok     

In [27]:
class chaii_ka_data(Dataset):
    def __init__(self, data_df, tokenizer, train=True):
        super(chaii_ka_data, self).__init__()
        '''
            train = True means train, train = False means val set; test = True means test set (without labels)
            data_df is the pandas dataframe containing context, question, ...        
        '''
        
        # tokenize data samples context;question, and create new samples if overflow
        # we need to do this apriori (and not in __getitem__ directly) because a datasample may create more samples upon tokenization
        self.reqd_keys = ['input_ids', 'attention_mask', 'start_positions', 'end_positions']
        self.data_tok = prepare_chaii(data_df, tokenizer)
    
    def __getitem__(self, input_id): # index is input_id as used in prepare_chaii()
        # sent = self.data_tok['input_ids'][input_id]
        # att_mask = self.data_tok['attention_mask'][input_id]
        # offset_map = self.data_tok['offset_mapping'][input_id]
        # start_idx_tok = self.data_tok['start_positions'][input_id]
        # end_idx_tok = self.data_tok['end_positions'][input_id]
        
        return {k: torch.tensor(v[input_id], dtype=torch.long) for k,v in self.data_tok.items() if k in self.reqd_keys}
    
    def __len__(self):
        return len(self.data_tok['input_ids'])

In [28]:
trainset = chaii_ka_data(train_df, tokenizer)
valset = chaii_ka_data(val_df, tokenizer)

### model

In [25]:
model = AutoModelForQuestionAnswering.from_pretrained(hyperparams.model_name)

### training

In [29]:
training_args = TrainingArguments(
    output_dir="ShAm-ki-chaii", 
    overwrite_output_dir=True, 
    per_device_eval_batch_size=hyperparams.batch_size,
    per_device_train_batch_size=hyperparams.batch_size,
    evaluation_strategy="epoch", 
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    seed=hyperparams.seed,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
#     metric_for_best_model="eval_accuracy",
#     greater_is_better=True,
#     warmup_steps=500,
#     logging_dir='./logs',
#     logging_strategy="epoch",
)

trainer = Trainer(
    model=model, args=training_args,
    train_dataset=trainset, eval_dataset=valset,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 11783
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2211


RuntimeError: CUDA out of memory. Tried to allocate 144.00 MiB (GPU 0; 10.92 GiB total capacity; 9.99 GiB already allocated; 143.38 MiB free; 10.01 GiB reserved in total by PyTorch)

### evaluation

### references
1. https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
2. https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.__call__

#### remove following code

In [74]:
# from datasets import load_dataset

In [75]:
# datasets = load_dataset("squad_v2")

In [53]:
# def prepare_train_features(examples):
#     max_length = 384
#     doc_stride = 128
#     pad_on_right = True
#     # Some of the questions have lots of whitespace on the left, which is not useful and will make the
#     # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
#     # left whitespace
#     examples["question"] = [q.lstrip() for q in examples["question"]]

#     # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
#     # in one example possible giving several features when a context is long, each of those features having a
#     # context that overlaps a bit the context of the previous feature.
#     tokenized_examples = tokenizer(
#         examples["question" if pad_on_right else "context"],
#         examples["context" if pad_on_right else "question"],
#         truncation="only_second" if pad_on_right else "only_first",
#         max_length=max_length,
#         stride=doc_stride,
#         return_overflowing_tokens=True,
#         return_offsets_mapping=True,
#         padding="max_length",
#     )

#     # Since one example might give us several features if it has a long context, we need a map from a feature to
#     # its corresponding example. This key gives us just that.
#     sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
#     # The offset mappings will give us a map from token to character position in the original context. This will
#     # help us compute the start_positions and end_positions.
#     offset_mapping = tokenized_examples.pop("offset_mapping")

#     # Let's label those examples!
#     tokenized_examples["start_positions"] = []
#     tokenized_examples["end_positions"] = []

#     for i, offsets in enumerate(offset_mapping):
#         # We will label impossible answers with the index of the CLS token.
#         input_ids = tokenized_examples["input_ids"][i]
#         cls_index = input_ids.index(tokenizer.cls_token_id)

#         # Grab the sequence corresponding to that example (to know what is the context and what is the question).
#         sequence_ids = tokenized_examples.sequence_ids(i)

#         # One example can give several spans, this is the index of the example containing this span of text.
#         sample_index = sample_mapping[i]
#         answers = examples["answers"][sample_index]
#         # If no answers are given, set the cls_index as answer.
#         if len(answers["answer_start"]) == 0:
#             tokenized_examples["start_positions"].append(cls_index)
#             tokenized_examples["end_positions"].append(cls_index)
#         else:
#             # Start/end character index of the answer in the text.
#             start_char = answers["answer_start"][0]
#             end_char = start_char + len(answers["text"][0])

#             # Start token index of the current span in the text.
#             token_start_index = 0
#             while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
#                 token_start_index += 1

#             # End token index of the current span in the text.
#             token_end_index = len(input_ids) - 1
#             while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
#                 token_end_index -= 1

#             # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
#             if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
#                 tokenized_examples["start_positions"].append(cls_index)
#                 tokenized_examples["end_positions"].append(cls_index)
#             else:
#                 # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
#                 # Note: we could go after the last offset if the answer is the last word (edge case).
#                 while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
#                     token_start_index += 1
#                 tokenized_examples["start_positions"].append(token_start_index - 1)
#                 while offsets[token_end_index][1] >= end_char:
#                     token_end_index -= 1
#                 tokenized_examples["end_positions"].append(token_end_index + 1)

#     return tokenized_examples

In [54]:
# tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

In [55]:
# print(datasets)
# print(datasets["train"].column_names)
# print(tokenized_datasets)

### data - understanding the pipeline

In [56]:
# tokenizer = AutoTokenizer.from_pretrained(hyperparams.tokenizer_name)

In [57]:
# train_df = pd.read_csv('train.csv', encoding='utf-8')
# test_df = pd.read_csv('test.csv')
# sample_df = pd.read_csv('sample_submission.csv')

In [58]:
# train_df = sklearn.utils.shuffle(train_df, random_state=4).reset_index(drop=True)

In [15]:
# # converting into Squad format
# def convert_answers(row):
#     return {'answer_start': [row[0]], 'text': [row[1]]}

# train_df['answers'] = train_df[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

In [59]:
# train_df = train_df.loc[:2]
# # train_df

In [60]:
# train_df

In [18]:
# for idx in range(len(train_df)):
#     if train_df.loc[idx,'language'] == "hindi" and len(tokenizer(train_df.loc[idx, 'context'], train_df.loc[idx, 'question'])['input_ids']) > hyperparams.max_len:
#         break

In [61]:
# out = tokenizer(
#     list(train_df['question']), list(train_df['context']),
#     max_length=hyperparams.max_len, 
#     truncation='only_second',
#     stride=hyperparams.overlap_len,
#     return_overflowing_tokens=True,
#     return_offsets_mapping=True,
#     padding=True,
# )

In [62]:
# print(out.keys())
# print(len(out['input_ids']), len(out['offset_mapping']))

In [63]:
# print(len(out['offset_mapping']))
# # for i in range(len(out['offset_mapping'])):
# #     print(len(out.sequence_ids(i)))

In [64]:
# print(tokenizer.cls_token, tokenizer.bos_token, tokenizer.padding_side)

In [65]:
# print(len(out['input_ids']))

In [24]:
# print(tokenizer.decode(out['input_ids'][0]))
# print(tokenizer.decode(out['input_ids'][1]))
# print(tokenizer.decode(out['input_ids'][2]))

In [66]:
# map_x2context_idx = out['overflow_to_sample_mapping']
# print(map_x2context_idx, type(map_x2context_idx))

In [67]:
# map_offset = out['offset_mapping']

In [68]:
# idx = random.randint(0, len(out['input_ids']))
# idx

In [69]:
# x_idx = out['input_ids'][idx]
# x_context_idx = map_x2context_idx[idx]
# print(idx, x_context_idx, len(x_idx))

In [70]:
# x_x1_x2 = out.sequence_ids(idx) # sequence ids

In [30]:
# tokenizer.decode(x_idx)

In [71]:
# answer_start = train_df.loc[x_context_idx, 'answer_start']
# answer = train_df.loc[x_context_idx, 'answer_text']
# print(answer)
# print(answer_start)

In [72]:
# cls_idx = x_idx.index(tokenizer.cls_token_id)
# cls_idx

In [73]:
# y_start_char = train_df.loc[x_context_idx, 'answer_start']
# y_end_char = train_df.loc[x_context_idx, 'answer_start'] + len(train_df.loc[x_context_idx, 'answer_text']) # note it is one char ahead

In [ ]:
# a = [(1,2.1)]
# torch.tensor(a, dtype=torch.long)

In [ ]:
# trainset = chaii_ka_data(train_df, tokenizer)
# next(iter(trainset))